In [1]:
import sys, os
import numpy as np
np.set_printoptions(threshold=np.inf)
import pandas as pd
from utils.utils import read_txt, read_txt_np_int
from functools import lru_cache
from math import ceil, floor
from copy import deepcopy

# INPUT

In [83]:
inputfilename = './inputs/day20.txt'

inputdata = read_txt(inputfilename)

testdata_small = ["###############",
"#...#...#.....#",
"#.#.#.#.#.###.#",
"#S#...#.#.#...#",
"#######.#.#.###",
"#######.#.#...#",
"#######.#.###.#",
"###..E#...#...#",
"###.#######.###",
"#...###...#...#",
"#.#####.#.###.#",
"#.#...#.#.#...#",
"#.#.#.#.#.#.###",
"#...#...#...###",
"###############"]

In [149]:
data_to_use = inputdata

race_chart = np.array([[char for char in line] for line in data_to_use])
race_chart

array([['#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#',
        '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#',
        '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#',
        '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#',
        '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#',
        '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#',
        '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#',
        '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#',
        '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#',
        '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#',
        '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#'],
       ['#', '#', '#', '.', '.', '.', '#', '.', '.', '.', '.', '.', '.',
        '.', '#', '.', '.', '.', '.', '.', '#', '.', '.', '.', '#', '.',
        '.', '.', '#', '.', '.', '.', '#', '.', '.', '.', '.

## PART 1

In [150]:
movement_list = [(-1, 0), (1, 0), (0, -1), (0, 1)]

mapping_values = {'.':0, '#':-1, 'S':0, 'E':0}
def print_route(race_chart):
    starting_position = (np.where(race_chart == 'S')[0][0], np.where(race_chart == 'S')[1][0])
    ending_position = (np.where(race_chart == 'E')[0][0], np.where(race_chart == 'E')[1][0])

    route_chart = np.array([[mapping_values[char] for char in line] for line in race_chart])

    path_distance = 1
    current_position = starting_position
    while True:
        #print(current_position)
        route_chart[current_position] = path_distance
        path_distance += 1
        if current_position == ending_position:
            break
        for movement in movement_list:
            next_position = (current_position[0] + movement[0], current_position[1] + movement[1])
            if next_position[0] < 0 or next_position[1] < 0:
                continue
            try:
                if route_chart[next_position] == 0:
                    break
            except IndexError:
                continue
        current_position = next_position
    return route_chart


def find_cheats(route_chart):
    cheat_list = []
    potential_cheat_starts = list(zip(np.where(route_chart >= 0)[0], np.where(route_chart >= 0)[1]))
    for cheat_start in potential_cheat_starts:
        original_distance = route_chart[cheat_start]
        for first_movement in movement_list:
            forbidden_step = cheat_start + np.array(first_movement)
            try:
                # Not a forbidden step -- skip
                if route_chart[forbidden_step[0], forbidden_step[1]] >= 0:
                    continue
            except IndexError:
                continue

            for second_movement in movement_list:
                back_on_track_step = forbidden_step + np.array(second_movement)
                try:
                    final_distance = route_chart[back_on_track_step[0], back_on_track_step[1]]
                    # If it is forbidden -- skip
                    if final_distance < 0:
                        continue
                    cheat_advantage = final_distance - original_distance - 2 # it takes 2 steps to complete the cheat
                    if cheat_advantage > 0:
                        cheat_list.append((cheat_start, back_on_track_step, cheat_advantage))
                except IndexError:
                    continue
    return cheat_list

In [151]:
# Solve part 1
route_chart = print_route(race_chart)
cheat_list = find_cheats(route_chart)

count_over_100 = 0
for cheat in cheat_list:
    count_over_100 += 1 if cheat[2] >= 100 else 0

cheat_list

[((np.int64(1), np.int64(7)), array([1, 5]), np.int64(36)),
 ((np.int64(1), np.int64(8)), array([3, 8]), np.int64(2)),
 ((np.int64(1), np.int64(9)), array([3, 9]), np.int64(4)),
 ((np.int64(1), np.int64(10)), array([ 3, 10]), np.int64(6)),
 ((np.int64(1), np.int64(11)), array([ 3, 11]), np.int64(8)),
 ((np.int64(1), np.int64(15)), array([ 1, 13]), np.int64(252)),
 ((np.int64(1), np.int64(16)), array([ 3, 16]), np.int64(2)),
 ((np.int64(1), np.int64(17)), array([ 3, 17]), np.int64(4)),
 ((np.int64(1), np.int64(21)), array([ 1, 19]), np.int64(4)),
 ((np.int64(1), np.int64(25)), array([ 1, 23]), np.int64(4)),
 ((np.int64(1), np.int64(29)), array([ 1, 27]), np.int64(4)),
 ((np.int64(1), np.int64(33)), array([ 1, 31]), np.int64(16)),
 ((np.int64(1), np.int64(34)), array([ 3, 34]), np.int64(2)),
 ((np.int64(1), np.int64(35)), array([ 3, 35]), np.int64(4)),
 ((np.int64(1), np.int64(41)), array([ 1, 39]), np.int64(192)),
 ((np.int64(1), np.int64(42)), array([ 3, 42]), np.int64(2)),
 ((np.int64

In [152]:
print(route_chart)

[[  -1   -1   -1   -1   -1   -1   -1   -1   -1   -1   -1   -1   -1   -1
    -1   -1   -1   -1   -1   -1   -1   -1   -1   -1   -1   -1   -1   -1
    -1   -1   -1   -1   -1   -1   -1   -1   -1   -1   -1   -1   -1   -1
    -1   -1   -1   -1   -1   -1   -1   -1   -1   -1   -1   -1   -1   -1
    -1   -1   -1   -1   -1   -1   -1   -1   -1   -1   -1   -1   -1   -1
    -1   -1   -1   -1   -1   -1   -1   -1   -1   -1   -1   -1   -1   -1
    -1   -1   -1   -1   -1   -1   -1   -1   -1   -1   -1   -1   -1   -1
    -1   -1   -1   -1   -1   -1   -1   -1   -1   -1   -1   -1   -1   -1
    -1   -1   -1   -1   -1   -1   -1   -1   -1   -1   -1   -1   -1   -1
    -1   -1   -1   -1   -1   -1   -1   -1   -1   -1   -1   -1   -1   -1
    -1]
 [  -1   -1   -1 8393 8392 8391   -1 8353 8352 8351 8350 8349 8348 8347
    -1 8093 8092 8091 8090 8089   -1 8083 8082 8081   -1 8075 8074 8073
    -1 8067 8066 8065   -1 8047 8046 8045 8044 8043 8042 8041   -1 7847
  7846 7845 7844 7843 7842 7841   -1 7819 7818 7817 7816

## PART 2

In [153]:
# NEED TO RUN PART 1 TO GET THE ROUTE CHART
def find_long_cheats(route_chart, max_cheat_length):
    cheat_list = []
    # Find all points in the route to attempt a cheat from there
    potential_cheat_starts = list(zip(np.where(route_chart >= 0)[0], np.where(route_chart >= 0)[1]))

    for cheat_idx, cheat_start in enumerate(potential_cheat_starts):
        print(f"Checking starting point {cheat_idx + 1} out of {len(potential_cheat_starts)}")
        original_distance = route_chart[cheat_start]

        positions_to_visit = set()
        visited_positions = set([cheat_start])

        # A cheat begins ALWAYS with an illegal move
        for first_movement in movement_list:
            current_position = cheat_start + np.array(first_movement)
            # Out of bounds
            if current_position[0] < 0 or current_position[1] < 0:
                continue
            try:
                # Not a forbidden step -- skip
                if route_chart[current_position[0], current_position[1]] >= 0:
                    pass # THIS SHOULD BE A CONTINUE BUT APPARENTLY NOT IN THE CURRENT PROBLEM CONCEPTION
                positions_to_visit.add(tuple(current_position))
                visited_positions.add(tuple(current_position))
            except IndexError:
                continue

        # Keep track of the positions visited to avoid repetition

        cheat_duration = 1

        # Iterate by cheat duration: at each step add all the new locations reachable in that time
        while cheat_duration < max_cheat_length:
            cheat_duration += 1
            next_positions_to_visit = set()

            while len(positions_to_visit) > 0:
                current_position = positions_to_visit.pop()

                for next_movement in movement_list:
                    next_position = tuple(current_position + np.array(next_movement))

                    # Out of bounds
                    if next_position[0] < 0 or next_position[1] < 0:
                        continue
                    # We do not repeat positions
                    if next_position in visited_positions:
                        continue

                    try:
                        next_distance = route_chart[next_position[0], next_position[1]]
                    except IndexError:
                        continue
                    visited_positions.add(next_position)
                    next_positions_to_visit.add(next_position)
                    cheat_advantage = next_distance - original_distance - cheat_duration
                    if cheat_advantage > 0:
                        cheat_list.append(([int(cheat_start[0]), int(cheat_start[1])], [int(next_position[0]), int(next_position[1])], int(cheat_advantage)))
            positions_to_visit = next_positions_to_visit
    
    return cheat_list

In [154]:
long_cheats = find_long_cheats(route_chart, 20)
long_cheats

Checking starting point 1 out of 9317
Checking starting point 2 out of 9317
Checking starting point 3 out of 9317
Checking starting point 4 out of 9317
Checking starting point 5 out of 9317
Checking starting point 6 out of 9317
Checking starting point 7 out of 9317
Checking starting point 8 out of 9317
Checking starting point 9 out of 9317
Checking starting point 10 out of 9317
Checking starting point 11 out of 9317
Checking starting point 12 out of 9317
Checking starting point 13 out of 9317
Checking starting point 14 out of 9317
Checking starting point 15 out of 9317
Checking starting point 16 out of 9317
Checking starting point 17 out of 9317
Checking starting point 18 out of 9317
Checking starting point 19 out of 9317
Checking starting point 20 out of 9317
Checking starting point 21 out of 9317
Checking starting point 22 out of 9317
Checking starting point 23 out of 9317
Checking starting point 24 out of 9317
Checking starting point 25 out of 9317
Checking starting point 26 out of 

[([1, 3], [5, 3], 4),
 ([1, 3], [5, 2], 2),
 ([1, 3], [6, 3], 4),
 ([1, 3], [7, 3], 4),
 ([1, 3], [7, 2], 4),
 ([1, 3], [7, 1], 4),
 ([1, 3], [9, 3], 8),
 ([1, 3], [9, 4], 8),
 ([1, 3], [8, 1], 4),
 ([1, 3], [9, 2], 6),
 ([1, 3], [9, 1], 4),
 ([1, 3], [11, 3], 32),
 ([1, 3], [9, 5], 8),
 ([1, 3], [10, 5], 8),
 ([1, 3], [12, 3], 30),
 ([1, 3], [11, 2], 32),
 ([1, 3], [11, 5], 8),
 ([1, 3], [11, 1], 32),
 ([1, 3], [13, 3], 28),
 ([1, 3], [12, 1], 32),
 ([1, 3], [12, 5], 8),
 ([1, 3], [14, 3], 26),
 ([1, 3], [13, 1], 32),
 ([1, 3], [13, 5], 8),
 ([1, 3], [15, 3], 24),
 ([1, 3], [14, 5], 8),
 ([1, 3], [14, 1], 32),
 ([1, 3], [16, 3], 22),
 ([1, 3], [15, 1], 32),
 ([1, 3], [15, 5], 8),
 ([1, 3], [17, 3], 20),
 ([1, 3], [17, 4], 18),
 ([1, 3], [15, 6], 8),
 ([1, 3], [16, 1], 32),
 ([1, 3], [15, 7], 8),
 ([1, 3], [17, 1], 32),
 ([1, 3], [19, 3], 40),
 ([1, 3], [17, 5], 16),
 ([1, 3], [20, 3], 38),
 ([1, 3], [19, 4], 40),
 ([1, 3], [17, 6], 14),
 ([1, 3], [15, 8], 8),
 ([1, 3], [18, 1], 32),
 

In [155]:

for saved_time in [100]:
    count = 0
    for long_cheat in long_cheats:
        if long_cheat[2] >= saved_time:
            count += 1
    if count:
        print(f"There are {count} cheats that save more than {saved_time} picoseconds")

There are 977747 cheats that save more than 100 picoseconds


In [139]:
# debuggin
saved_time = 72
for long_cheat in long_cheats:
    if long_cheat[2] == saved_time:
        print(long_cheat)


([1, 2], [7, 3], 72)
([1, 2], [7, 4], 72)
([1, 2], [7, 5], 72)
([1, 3], [7, 4], 72)
([1, 3], [7, 5], 72)
([2, 1], [8, 3], 72)
([2, 3], [7, 4], 72)
([2, 3], [7, 5], 72)
([3, 1], [9, 1], 72)
([3, 1], [9, 2], 72)
([3, 1], [9, 3], 72)
([3, 3], [7, 3], 72)
([3, 3], [7, 4], 72)
([3, 3], [7, 5], 72)
([3, 4], [7, 4], 72)
([3, 4], [7, 5], 72)
([3, 5], [7, 5], 72)
